# Analyzing Euro 2024 Soccer Data in SQL

Today, we are going to explore some data of the EURO 2024 which is still ongoing at the time of writing and working through this.
You will work with a dataset that was sourced from the `sportmonks` API (with some adjustments)


### Problem Context:

**Your boss** is invited to garden party of **his boss who is a huge soccer fan**. However, **your boss has no idea about soccer**. He would like to have at least a few interesting **facts about EURO 2024** that he can mention so that **he doesn't look like a complete loser** during the conversations.



**We will cover the following**

- 🕵️‍♀️ Exploratory data analysis --> How does the data look like and can we discover potential issues?
- 🔧 Data modeling --> Bring the data into a format that makes it easier for us to analyze
- 🤿 Deep dive analysis: Game events and goals
- 📊 Based on what we found - see how things could be monitored in a [EURO 2024 dashboard in Metabase](https://www.metabase.com/gallery/uefa-euro-2024-stats-dashboard?utm_source=datacamp&utm_medium=post&utm_campaign=euro-2024-dashboard)



Mixture of walk through + tasks for you 🫵

## Task 0: Setup 🛠️

<details>
    <summary>Details</summary>
  <p>
      
For this analysis we only need the CSV files with some data from the EURO 2024 tournament.
      
* `game_events.csv` -> Events that happened during a game
      
Make sure that you have the CSV files mentioned above in your environment.  
We will use datalab's duckdb integration to analyze the files with SQL.

_The data is sourced and adjusted from [sportmonks](https://docs.sportmonks.com/football)_
      
  </p>
</details>

## Task 1:  Get an understanding of your AUDIENCE and what you want to achieve

Before diving into a dataset, we should set ourselves a clear goal on what we want to achive.   
Otherwise, we might end in some rabbit holes easily 🐇🕳️.


In this session we want to understand more about goals since this is what everyone talks about at a garden party (or not?) 🪴

* When are they scored?
* Commentators often talk about "jokers" that substituted late in the game -> Can we see some effects in the data?


## Task 2: Have a brief look at the data 👀

When we look at the data we try to already get a glimbse of it, e.g.:

* What data (columns) do we have available? Do we know what they are?
* Can we already see a row count?
* Missing values and where they occur

In [1]:
SELECT * FROM 'game_events.csv' LIMIT 5;

,id,game,event_type,created_at,team_name,player_name,related_player_name,result,minute,extra_minute,previous_player_event,previous_player_event_at,seconds_after_previous_event,player_id,related_player_id,participant_id,fixture_id
0,116213644,Romania vs Ukraine (2024-06-17),Substitution,2024-06-17 16:27:00+00:00,Romania,Bogdan Racovițan,Nicolae Stanciu,None,87,0,None,NaT,NaN,31616427,62905.0,18638,19032635
1,116213524,Romania vs Ukraine (2024-06-17),Substitution,2024-06-17 16:02:00+00:00,Ukraine,Volodymyr Brazhko,Taras Stepanenko,None,62,0,None,NaT,NaN,24065258,204483.0,18624,19032635
2,116213531,Romania vs Ukraine (2024-06-17),Substitution,2024-06-17 16:02:00+00:00,Romania,Valentin Mihăilă,Florinel Coman,None,62,0,None,NaT,NaN,6013442,165479.0,18638,19032635
3,116213571,Romania vs Ukraine (2024-06-17),Yellowcard,2024-06-17 16:07:00+00:00,Ukraine,Yukhym Konoplya,None,None,67,0,None,NaT,NaN,4545319,NaN,18624,19032635
4,116213583,Romania vs Ukraine (2024-06-17),Substitution,2024-06-17 16:12:00+00:00,Ukraine,Oleksandr Tymchyk,Yukhym Konoplya,None,72,0,None,NaT,NaN,206981,4545319.0,18624,19032635


| Field               | Description                                         | Type          |
|---------------------|-----------------------------------------------------|---------------|
| id                  | Refers to the unique id of the type                | integer       |
| game          | The name of the game          | string       |
| event_type          | The type of event   | string       |
| created_at          | The time when the event happened         | timestamp       |
| team_name          | The name of the team related to the event      | string       |
| player_name          | The name of the player that performed the event or to whom it happened          | string       |
| related_player_name          | The name of the player who is related to the event that `player_name` performed          | string       |
| result          | The result after the event happend    | string       |
| minute          | The game minute in which the event happened        | integer       |
| extra_minute    | The additional minutes in which the event happened (e.g. for 90+2 this would be 2)         | integer       |
| previous_player_event    | The `event_type` of the previous event that happened to the player        | string       |
| previous_player_event_at | The timestamp of the previous event that happened to the player        | timestamp       |
| seconds_after_previous_event | The seconds after the previous event happened to the player        | integer       |
| player_id          | The id of the player that performed the event or to whom it happened       | integer       |
| related_player_id          | The id of the player who is related to the event that `player_name` performed          | integer       |
| participant_id          | The id of the team related to the event          | integer       |
| fixture_id          | The id of the game the event happened in      | integer       |

## Task 3: 🕵️‍♀️ Exploratory data analysis

Now, we want to further understand the data we work with and potential issues it has.  
For that, a good start is to look at
* Row count
* Duplicates
* Missing values
* Value distributions (i.e. can we see any weird outliers)

### Row counts and uniqueness
Let's start by looking at the row counts

In [2]:
SELECT 
	COUNT(*) AS rows,
	COUNT(DISTINCT id) AS unique_ids
FROM 'game_events.csv';

,rows,unique_ids
0,838,838


### Summary stats
Let's look at the separate tables and their columns using the `SUMMARIZE` function.

In [3]:
SUMMARIZE SELECT * FROM 'game_events.csv'

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,BIGINT,116112635,116635472,831,116364186.64319809,148824.0936745581,116244440,116356131,116489138,838,0.0%
1,game,VARCHAR,Albania vs Spain (2024-06-24),Ukraine vs Belgium (2024-06-26),48,None,None,None,None,None,838,0.0%
2,event_type,VARCHAR,Goal,Yellowcard,9,None,None,None,None,None,838,0.0%
3,created_at,TIMESTAMP,2024-06-14 21:10:00,2024-07-06 22:36:00,624,None,None,None,None,None,838,0.0%
4,team_name,VARCHAR,Albania,Ukraine,24,None,None,None,None,None,838,0.0%
5,player_name,VARCHAR,Abdülkerim Bardakcı,Ž. Karničnik,413,None,None,None,None,None,838,0.0%
6,related_player_name,VARCHAR,A. Bah,İsmail Yüksek,296,None,None,None,None,None,838,37.11%
7,result,VARCHAR,0-0,5-3,24,None,None,None,None,None,838,84.25%
8,minute,BIGINT,1,120,108,64.85202863961814,24.88769642123774,52,70,83,838,0.0%
9,extra_minute,BIGINT,0,11,12,0.32577565632458233,1.250119824996759,0,0,0,838,0.0%


### Distributions

In [5]:
-- Distribution of "minute" values
SELECT 
	minute, 
	COUNT(1) AS events
FROM 'game_events.csv'
GROUP BY 1
ORDER BY 1;

,minute,events
0,1,5
1,2,5
2,3,4
3,4,4
4,5,4
...,...,...
105,115,1
106,117,1
107,118,3
108,119,2


* Seems that most events happen in the 2nd half of the game
* There are many events in the 90th minute - are those outliers or data issues?

Let's check what type of events happen in the 90th minute.

In [8]:
-- Events in minute 90
SELECT 
	event_type, 
	COUNT(1) AS events 
FROM 'game_events.csv'
WHERE minute = 90
GROUP BY event_type
ORDER BY events DESC;

,event_type,events
0,Yellowcard,32
1,Substitution,29
2,Goal,12
3,Redcard,2
4,VAR,1


### 🫵 Your turn: Distribution of `extra_minute`

Try to create a similar plot as above but this time we look at the distribution of `extra_minute`.  
Before running your query: What would you expect and why?

In [10]:
-- Distribution of "extra_minute"
SELECT 
	extra_minute,
    COUNT(1) as events
FROM 'game_events.csv'
GROUP BY 1
ORDER BY extra_minute;

,extra_minute,events
0,0,763
1,1,13
2,2,16
3,3,13
4,4,12
5,5,7
6,6,5
7,7,2
8,8,4
9,9,1


## Task 4: Data modelling for easier analysis

Modelling data by joining tables and pre-calculating metrics for data analytics purposes can be good investment upfront to save time down the road. Let's try to create some data models that can be useful for our analysis.

### Hold on - First, let's cover some special things
First, we cover some special functions that we are going to use.

Let's look at the [lpad function](https://duckdb.org/docs/sql/functions/char#lpadstring-count-character)

In [13]:
-- lpad function: Pads the `string` with the `character` from the left until it has `count` characters.
-- Syntax:  lpad(string, count, character)
SELECT 
	lpad(1, 1, '0'),
	lpad(1, 2, '0'),
	lpad(1, 3, '0'),
	lpad('01', 2, '0'),
	lpad(10, 2, '0'),
	lpad(100, 2, '0'), -- WARNING! THIS IS TRIMMED TO "10"

,"lpad(1, 1, '0')","lpad(1, 2, '0')","lpad(1, 3, '0')","lpad('01', 2, '0')","lpad(10, 2, '0')","lpad(100, 2, '0')"
0,1,01,001,01,10,10


--> This is useful when you want to make a number string sortable

In [11]:
WITH data as (
	SELECT '1' as col
	UNION ALL
	SELECT '5' as col
	UNION ALL
	SELECT '10' as col
	UNION ALL
	SELECT '100' as col
)

SELECT * FROM data
ORDER BY col

,col
0,1
1,10
2,100
3,5


In [14]:
-- Prefixing the numbers with empty strings using lpad helps fix the ordering
WITH data as (
	SELECT lpad('1', 3, ' ') as col
	UNION ALL
	SELECT lpad('5', 3, ' ') as col
	UNION ALL
	SELECT lpad('10', 3, ' ') as col
	UNION ALL
	SELECT lpad('100', 3, ' ') as col
)

SELECT * FROM data
ORDER BY col

,col
0,1
1,5
2,10
3,100


Now, a small trick for creating histograms

In [15]:
/*
We can easily bucket numeric values for histograms using this approach:

ceil(<field> / <bucket_size>) * <bucket_size> as ceil_bucket
*/

-- Example: Bucket size 5 minutes
WITH buckets AS (
	SELECT DISTINCT
		minute,
		minute / 5,
		ceil(minute / 5),
		ceil(minute / 5) * 5 as ceil_bucket
	FROM 'game_events.csv'
	ORDER BY minute
)

SELECT * FROM buckets

,minute,"(""minute"" / 5)","ceil((""minute"" / 5))",ceil_bucket
0,1,0.2,1.0,5.0
1,2,0.4,1.0,5.0
2,3,0.6,1.0,5.0
3,4,0.8,1.0,5.0
4,5,1.0,1.0,5.0
...,...,...,...,...
105,115,23.0,23.0,115.0
106,117,23.4,24.0,120.0
107,118,23.6,24.0,120.0
108,119,23.8,24.0,120.0


### Now let's create our data model

What we want to fix:

* `minute` and `extra_minute` on their own are not really usable. Let's combine them to a `game_minute` categories ( e.g. `90 + 5`)
* We don't need to know in which exact minute an event happened but rather in which time ranges. Let's bucket the game minutes into buckets of 5 minutes

In [2]:
COPY (
	WITH goal_events_with_minute_string as (
		SELECT 
			*,							
			-- Use lpad for better sorting behavior (e.g. 1 -> '  1')
			lpad(minute, 3, ' ') as minute_str_sortable,
		FROM 'game_events.csv'
		WHERE event_type = 'Goal'
	)

	SELECT 
		*,
		CASE 
			WHEN extra_minute = 0 THEN minute_str_sortable
			ELSE concat(minute_str_sortable, ' + ', extra_minute)
		END AS game_minute,
		CASE
			-- We simplify overtime and move it into a "OT" bucket
			WHEN minute > 90 THEN 'OT'
			-- For extra minutes, we use 45+, 90+, etc.
			WHEN extra_minute != 0 THEN concat(minute_str_sortable, ' +')
	 		-- Use lpad for better sorting behavior (e.g. 1 -> '  1')
			ELSE lpad(
				(ceil(minute / 5) * 5)::int,
				3,
				' '
			)
		END AS game_period_bucket
	FROM goal_events_with_minute_string
) TO 'goals.csv' (HEADER, DELIMITER ',');

SELECT * FROM 'goals.csv'

,id,game,event_type,created_at,team_name,player_name,related_player_name,result,minute,extra_minute,previous_player_event,previous_player_event_at,seconds_after_previous_event,player_id,related_player_id,participant_id,fixture_id,minute_str_sortable,game_minute,game_period_bucket
0,116213507,Romania vs Ukraine (2024-06-17),Goal,2024-06-17 15:57:00+00:00,Romania,Denis Drăguș,Dennis Man,3-0,57,0,None,NaT,NaN,165648,165622.0,18638,19032635,57,57,60
1,116213493,Romania vs Ukraine (2024-06-17),Goal,2024-06-17 15:53:00+00:00,Romania,Răzvan Marin,None,2-0,53,0,None,NaT,NaN,64202,NaN,18638,19032635,53,53,55
2,116213437,Romania vs Ukraine (2024-06-17),Goal,2024-06-17 15:29:00+00:00,Romania,Nicolae Stanciu,Dennis Man,1-0,29,0,None,NaT,NaN,62905,165622.0,18638,19032635,29,29,30
3,116479263,Germany vs Denmark (2024-06-29),Goal,2024-06-29 22:08:00+00:00,Germany,Jamal Musiala,Nico Schlotterbeck,2-0,68,0,None,NaT,NaN,33186829,3862352.0,18660,19032605,68,68,70
4,116478811,Germany vs Denmark (2024-06-29),Goal,2024-06-29 21:53:00+00:00,Germany,Kai Havertz,None,1-0,53,0,None,NaT,NaN,32612,NaN,18660,19032605,53,53,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,116182305,Poland vs Netherlands (2024-06-16),Goal,2024-06-16 15:16:00+00:00,Poland,Adam Buksa,Piotr Zieliński,1-0,16,0,None,NaT,NaN,155460,129072.0,18699,19032628,16,16,20
104,116182567,Poland vs Netherlands (2024-06-16),Goal,2024-06-16 15:29:00+00:00,Netherlands,Cody Gakpo,Nathan Aké,1-1,29,0,None,NaT,NaN,30062,1869.0,18694,19032628,29,29,30
105,116184468,Poland vs Netherlands (2024-06-16),Goal,2024-06-16 16:23:00+00:00,Netherlands,Wout Weghorst,Nathan Aké,1-2,83,0,Substitution,2024-06-16 16:21:00+00:00,120.0,25317,1869.0,18694,19032628,83,83,85
106,116632419,England vs Switzerland (2024-07-06),Goal,2024-07-06 19:20:00+00:00,England,Bukayo Saka,Declan Rice,1-1,80,0,Penalty Shootout Goal,2024-07-06 18:05:00+00:00,4500.0,16827155,5273.0,18645,19032604,80,80,80


## Task 5: Deep dive into goals

### 🫵 Your turn

#### When are they scored?

Count the number of goals per `game_period_bucket` and create a bar chart using the `goals.csv` table.

Bonus: Figure out what the fastest goal was

In [3]:
WITH counts as (
	SELECT 
        game_period_bucket,
        COUNT(1) as goals
    FROM 'goals.csv'
    GROUP BY 1
    ORDER BY 1
)

SELECT * FROM counts

,game_period_bucket,goals
0,5,4
1,10,4
2,15,6
3,20,8
4,25,5
5,30,7
6,35,4
7,40,4
8,45,1
9,45 +,3


#### Fastest goal

In [7]:
WITH fastest_goals AS (
	SELECT 
		game, 
		player_name, 
		game_minute
	FROM 'goals.csv'
	ORDER BY game_minute ASC
)

SELECT * FROM fastest_goals LIMIT 5;

,game,player_name,game_minute
0,Austria vs Turkey (2024-07-02),Merih Demiral,1
1,Italy vs Albania (2024-06-15),Nedim Bajrami,1
2,Georgia vs Portugal (2024-06-26),Khvicha Kvaratskhelia,2
3,Belgium vs Romania (2024-06-22),Youri Tielemans,2
4,Netherlands vs Austria (2024-06-25),Donyell Malen,6


### "Jokers" and their impact

Often coaches use the term "joker" for some of their players.   
They usually use that for players they can substitute and they will likely score a goal.  
Let's see how many of those goals we can actually find.

Let's classify a goal as "joker goal" if:

* The player's last event was a substitution 
* The goal happened within 15 minutes (900s) after the substitution

In [8]:
WITH counts AS (
	SELECT 
		game_period_bucket,
		CASE
			-- Joker goal if player was substituted within 15 minutes prior to scoring
			WHEN 
				previous_player_event = 'Substitution' 
				AND seconds_after_previous_event <= 900 
			THEN 'joker goal'
			ELSE 'normal goal'
		END AS goal_type,
		COUNT(1) AS cnt
	FROM 'goals.csv'
	GROUP BY 1, 2
	ORDER BY 1
)

SELECT
	*
FROM counts

,game_period_bucket,goal_type,cnt
0,5,normal goal,4
1,10,normal goal,4
2,15,normal goal,6
3,20,normal goal,8
4,25,normal goal,5
5,30,normal goal,7
6,35,normal goal,4
7,40,normal goal,4
8,45,normal goal,1
9,45 +,normal goal,3


## Bonus: 📊 Use a dashboard to monitor the EURO 2024 results

Here is an example dashboard build with Metabase (an easy to use OSS BI tool):

[https://www.metabase.com/gallery/uefa-euro-2024-stats-dashboard](https://www.metabase.com/gallery/uefa-euro-2024-stats-dashboard?utm_source=datacamp&utm_medium=post&utm_campaign=euro-2024-dashboard)


🎓 There will also be another [webinar](https://www.metabase.com/events/euro-2024-dashboard) (2024-07-23) on how we build this where you can learn more about building dashboards.

### What game had the most goals?

In [10]:
SELECT 
	game,
	COUNT(*) as goals
FROM 'goals.csv'
GROUP BY game
ORDER BY goals DESC
LIMIT 1;

,game,goals
0,Germany vs Scotland (2024-06-14),6


### How many goals are there usually per game?

Try to create a bar plot that shows the distribution of goals per game.  
Use this template to get started:


```sql
WITH goals_per_game as (
	select 
		fixture_id,
		count(1) as goals
	from 'tmp_goals.csv'
	group by 1
	order by 1
)

select
	goals, 
    COUNT(1) AS cnt
from goals_per_game
GROUP BY 1
ORDER BY 2
```